In [34]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, silhouette_score
from sklearn.decomposition import PCA
import re
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

In [35]:
data=pd.read_csv('raw_data.csv')
com=pd.read_csv('data.csv')
features=pd.read_csv('features.csv')

In [36]:
def I_label(score):
    if(score==6 or score==7):
        return 0
    if(score>=2 and score <=5):
        return 1
    else:
        return 2
def C_label(score):
    if(score>=4 and score<=7):
        return 0
    elif(score==2 or score==3):
        return 1
    else:
        return 2
def A_label(score):
    if(score>=4 and score<=6):
        return 0
    if(score>=7 and score <=9):
        return 1
    else:
        return 2
def AM_LC_label(score):
    if(score>=4 and score<=5):
        return 0
    elif(score>=2.6 and score<=3.9):
        return 1
    else:
        return 2
def rest_label(score):
    if(score>=4 and score<=6):
        return 0
    elif(score>=2.6 and score<=3.9):
        return 1
    else:
        return 2
def EN_label(score):
    if(score>=0 and score<=25):
        return 0
    elif(score>=26 and score<=40):
        return 1
    else:
        return 2
def AO_label(score):
    if(score>=0 and score<=30):
        return 0
    elif(score>=31 and score<=40):
        return 1
    else:
        return 2
def C3_label(score):
    if(score>=36 and score<=50):
        return 0
    elif(score>=26 and score<=35):
        return 1
    else:
        return 2

In [37]:
mapping={
    'most people':3,
    'many people':2,
    'some people':1,
    'a few people':0,
    'one or two people':-1,
    'nobody':-2,
    'usually': 3,
    'often':2,
    'sometimes':1,
    'occasionally':0,
    'rarely':-1,
    'never':-2,
    'disagree':-2,
    'slightly disagree':-1,
    'neutral':0,
    'slightly agree':1,
    'agree':2
}

In [38]:
def find_max(A):
    maxn=0
    (row,col)=(-1,-1)
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if(A[i][j]>=maxn):
                maxn=A[i][j]
                (row,col)=(i,j)
    return (row,col)
def pick_features(matrix,code,threshold):
    row,col=find_max(matrix)
    val=matrix[row][col]
    selected=[]
    while(val>=threshold and row>=0 and col>=0):
        maxn1=0
        for i in range(matrix.shape[0]):
            if(i!=col):
                temp=matrix[row][i]
                if(temp>maxn1):
                    maxn1=temp
        maxn2=0
        for i in range(matrix.shape[0]):
            if(i!=row):
                temp=matrix[col][i]
                if(temp>maxn2):
                    maxn2=temp
        if(maxn1<maxn2):
            selected.append(code[row])
        else:
            selected.append(code[col])
        matrix=np.delete(matrix,[row,col],axis=0)
        matrix=np.delete(matrix,[row,col],axis=1)
        newlist=[]
        for i in range(len(code)):
            if(i!=row and i!=col):
                newlist.append(code[i])
        code=newlist[:]
        row,col=find_max(matrix)
        if(row>=0 and col>=0):
            val=matrix[row][col]
    for c in code:
        selected.append(c)
    return selected

In [39]:
def features_test13(trait,start,num_features,feature_mapping):
    codelist=[]
    y=[]
    for i in range(start,start+num_features):
        codelist.append(features['Code'][i])
    df=data[data['Code'].isin(codelist)]
    to_delete=[]
    cnt=0
    for i in df.index:
        x=[]
        for column in df.columns[2:]:
            val=df[column][i]
            if not pd.isna(val):
                res = re.sub(' +', ' ',val)
                res=res.lower()
                res=''.join(c for c in res if (c.isalpha()or c==' '))
                res=res.strip()
                if(res in feature_mapping):
                    x.append(feature_mapping[res])
                else:
                    x.append(-1)
                    to_delete.append(cnt)
                    cnt=cnt+1
        y.append(x)
    new_y=[]
    for x in y:
        x_new=[]
        for i in range(len(x)):
            if(i not in to_delete):
                x_new.append(x[i])
        new_y.append(x_new)
    y=np.array(new_y)
    corr_matrix=[]
    for i in range(y.shape[0]):
        row=[]
        x1=y[i]
        for j in range(y.shape[0]):
            if(i==j):
                row.append(0)
                continue
            x2=y[j]
            row.append(abs(np.corrcoef(x1,x2)[0][1]))
        corr_matrix.append(row)
    corr_matrix=np.array(corr_matrix)
    featureslist=pick_features(corr_matrix,codelist,0.3)
    return featureslist

In [40]:
def features_test2(trait,start,num_features):
    codelist=[]
    y=[]
    for i in range(start,start+num_features):
        codelist.append(features['Code'][i])
    df=data[data['Code'].isin(codelist)]
    for i in df.index:
        x=[]
        for column in df.columns[2:]:
            val=df[column][i]
            x.append(int(val))
        y.append(x)
    y=np.array(y)
    corr_matrix=[]
    for i in range(y.shape[0]):
        row=[]
        x1=y[i]
        for j in range(y.shape[0]):
            if(i==j):
                row.append(0)
                continue
            x2=y[j]
            row.append(abs(np.corrcoef(x1,x2)[0][1]))
        corr_matrix.append(row)
    corr_matrix=np.array(corr_matrix)
    featureslist=pick_features(corr_matrix,codelist,0.3)
    return featureslist

In [41]:
questions=[]
questions=questions+features_test13('EI',0,9,mapping)
questions=questions+features_test13('WI',9,9,mapping)
questions=questions+features_test13('EC',18,9,mapping)
questions=questions+features_test13('WC',27,9,mapping)
questions=questions+features_test13('EA',36,9,mapping)
questions=questions+features_test13('WA',45,9,mapping)
questions=questions+features_test13('E',74,10,mapping)
questions=questions+features_test13('A',84,10,mapping)
questions=questions+features_test13('C',94,10,mapping)
questions=questions+features_test13('N',104,10,mapping)
questions=questions+features_test13('O',114,10,mapping)
questions=questions+features_test2('AM',54,3)
questions=questions+features_test2('LC',57,4)
questions=questions+features_test2('MC',61,4)
questions=questions+features_test2('ND',65,2)
questions=questions+features_test2('PfW',67,3)
questions=questions+features_test2('SE',70,4)

In [42]:
Y=[int(x) for x in list(com['Final Label'])]

In [43]:
mean=[]
for i in data.index:
    s=0
    c=0
    for column in data.columns[2:]:
        val=data[column][i]
        res = re.sub(' +', ' ',val)
        res=res.lower()
        res=''.join(c for c in res if (c.isalpha()or c==' '))
        res=res.strip()
        if(i>=54 and i<=73 and not pd.isna(val)):
            s=s+int(val)
            c=c+1
        elif(res in mapping):
            s=s+mapping[res]
            c=c+1
    if(c==0):
        print(data.loc[i])
    s=s/c
    mean.append(s)

In [44]:
indices=[]
for i in data.index:
    if(data['Code'][i] in questions):
        indices.append(i)
print(indices)

[0, 1, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 16, 17, 20, 24, 27, 28, 29, 32, 33, 34, 35, 36, 37, 41, 42, 43, 47, 51, 53, 54, 55, 56, 57, 58, 59, 62, 63, 64, 68, 72, 73, 74, 75, 76, 78, 79, 80, 81, 82, 83, 85, 86, 87, 88, 89, 92, 93, 94, 95, 97, 100, 101, 104, 107, 109, 110, 113, 114, 115, 118, 119, 121, 122, 123]


In [45]:
def compute_features(df,feature_mapping):
    X=[]
    for column in df.columns[2:]:
        values=list(df[column])
        l=len(values)
        x=[]
        for i in range(l):
            if(i in indices):
                value=values[i]
                res = re.sub(' +', ' ',value)
                res=res.lower()
                res=''.join(y for y in res if (y.isalpha()or y==' '))
                res=res.strip()
                if(i>=54 and i<=73 and not pd.isna(value)): 
                    x.append(int(value))
                elif(res in feature_mapping):
                    x.append(feature_mapping[res])
                else:
                    x.append(mean[i])
        X.append(x)
    return np.array(X)

In [46]:
X=compute_features(data,mapping)

In [47]:
X.shape

(4546, 76)

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
svm_classifier = SVC(kernel='rbf', C=1.0, random_state=42)
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_train)
print("Support Vector Machines: ")
print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
y_pred = svm_classifier.predict(X_test)
print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))
random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_classifier.fit(X_train, y_train)
y_pred = random_forest_classifier.predict(X_train)
print("Random Forest Classifier :")
print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
y_pred = random_forest_classifier.predict(X_test)
print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))
logistic_regression_classifier = LogisticRegression(max_iter=1000, random_state=42)
logistic_regression_classifier.fit(X_train, y_train)
y_pred = logistic_regression_classifier.predict(X_train)
print("Logistic Regression Classifier :")
print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
y_pred = logistic_regression_classifier.predict(X_test)
print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))
mlp = MLPClassifier(hidden_layer_sizes=(100,50), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_train)
print("Multi Layer Perceptron :")
print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
y_pred = mlp.predict(X_test)
print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))

Support Vector Machines: 
Training accuracy: 0.8037154729894891
Testing accuracy: 0.8351648351648352
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.8263736263736263
Logistic Regression Classifier :
Training accuracy: 0.6871180640430212
Testing accuracy: 0.7318681318681318
Multi Layer Perceptron :
Training accuracy: 0.9946223417257394
Testing accuracy: 0.7494505494505495


In [51]:
def cluster_distance(i,cluster,distance):
    d=0
    k=len(cluster)
    if(k==0):
        return np.inf
    for c in cluster:
        d=d+distance[i][c]
    return (d/k)
def assign_cluster(i,clusters,distance):
    assign=-1
    minn=np.inf
    j=0
    for cluster in clusters:
        d=cluster_distance(i,cluster,distance)
        if(d<minn):
            minn=d
            assign=j
        j=j+1
    return assign
def cluster_features(n,k,distance):
    labels=np.random.randint(0,k,size=n)
    clusters=[[] for i in range(k)]
    for i in range(n):
        l=labels[i]
        clusters[l].append(i)
    cnt=0
    while(True):
        cnt=cnt+1
        new_labels=np.zeros(n,dtype=int)
        for i in range(n):
            new_labels[i]=assign_cluster(i,clusters,distance)
        new_clusters=[[] for i in range(k)]
        for i in range(n):
            labels[i]=new_labels[i]
            new_clusters[new_labels[i]].append(i)
        to_remove=[]
        for i in range(k):
            if(len(new_clusters[i])==0):
                to_remove.append(i)
        new_clusters = [element for index, element in enumerate(new_clusters) if index not in to_remove]
        k=k-len(to_remove)
        s1=set()
        for i in clusters:
            s1.add(tuple(i))
        s2=set()
        for i in new_clusters:
            s2.add(tuple(i))
        if(s1==s2 or cnt>=100):
            break
        clusters=list(new_clusters)
    return (labels,clusters)
def silhouette(labels,clusters,distance):
    n=len(labels)
    k=len(clusters)
    sill=0
    for i in range(n):
        c=labels[i]
        d=0
        for point in clusters[c]:
            d=d+distance[i][point]
        a=d/len(clusters[c])
        minn=np.inf
        for j in range(k):
            if(c!=j):
                d=cluster_distance(i,clusters[j],distance)
                if(d<minn):
                    minn=d
        b=minn
        s=(b-a)/max(a,b)
        sill=sill+s
    return (sill/n)

In [69]:
def reduced_test2(trait,start,num_features):
    codelist=[]
    y=[]
    for i in range(start,start+num_features):
        codelist.append(features['Code'][i])
    df=data[data['Code'].isin(codelist)]
    for i in df.index:
        x=[]
        for column in df.columns[2:]:
            val=df[column][i]
            x.append(int(val))
        y.append(x)
    y=np.array(y)
    dist_matrix=[]
    for i in range(y.shape[0]):
        row=[]
        x1=y[i]
        for j in range(y.shape[0]):
            x2=y[j]
            row.append(abs(np.corrcoef(x1,x2)[0][1]))
        dist_matrix.append(row)
    dist_matrix=np.array(dist_matrix)
    maxn=-1
    k=1
    for i in range(2,10):
        labels,clusters=cluster_features(num_features,i,dist_matrix)
        sill=silhouette(labels,clusters,dist_matrix)
        if(sill>maxn):
            maxn=sill
            k=i
    labels,clusters=cluster_features(num_features,k,dist_matrix)
    xmax=0
    i=-1
    j=0
    for cluster in clusters:
        if(len(cluster)>xmax):
            xmax=len(cluster)
            i=j
        j=j+1 
    newlist=[]
    for c in clusters[i]:
        newlist.append(codelist[c])
    return newlist

In [70]:
def reduced_test13(trait,start,num_features,feature_mapping):
    codelist=[]
    y=[]
    for i in range(start,start+num_features):
        codelist.append(features['Code'][i])
    df=data[data['Code'].isin(codelist)]
    to_delete=[]
    cnt=0
    for i in df.index:
        x=[]
        for column in df.columns[2:]:
            val=df[column][i]
            if not pd.isna(val):
                res = re.sub(' +', ' ',val)
                res=res.lower()
                res=''.join(c for c in res if (c.isalpha()or c==' '))
                res=res.strip()
                if(res in feature_mapping):
#                     if(df['Code'][i] in negative_questions):
#                         x.append(-1*feature_mapping[res])
#                     else:
                    x.append(feature_mapping[res])
                else:
                    x.append(-1)
                    to_delete.append(cnt)
                    cnt=cnt+1
        y.append(x)
    new_y=[]
    for x in y:
        x_new=[]
        for i in range(len(x)):
            if(i not in to_delete):
                x_new.append(x[i])
        new_y.append(x_new)
    y=np.array(new_y)
    dist_matrix=[]
    for i in range(y.shape[0]):
        row=[]
        x1=y[i]
        for j in range(y.shape[0]):
            x2=y[j]
            row.append(abs(np.corrcoef(x1,x2)[0][1]))
        dist_matrix.append(row)
    dist_matrix=np.array(dist_matrix)
    maxn=-1
    k=1
    for i in range(2,10):
        labels,clusters=cluster_features(num_features,i,dist_matrix)
        sill=silhouette(labels,clusters,dist_matrix)
        if(sill>maxn):
            maxn=sill
            k=i
    labels,clusters=cluster_features(num_features,k,dist_matrix)
    xmax=0
    i=-1
    j=0
    for cluster in clusters:
        if(len(cluster)>xmax):
            xmax=len(cluster)
            i=j
        j=j+1 
    newlist=[]
    for c in clusters[i]:
        newlist.append(codelist[c])
    return newlist

In [71]:
questions=[]
questions=questions+reduced_test13('EI',0,9,mapping)
questions=questions+reduced_test13('WI',9,9,mapping)
questions=questions+reduced_test13('EC',18,9,mapping)
questions=questions+reduced_test13('WC',27,9,mapping)
questions=questions+reduced_test13('EA',36,9,mapping)
questions=questions+reduced_test13('WA',45,9,mapping)
questions=questions+reduced_test13('E',74,10,mapping)
questions=questions+reduced_test13('A',84,10,mapping)
questions=questions+reduced_test13('C',94,10,mapping)
questions=questions+reduced_test13('N',104,10,mapping)
questions=questions+reduced_test13('O',114,10,mapping)
questions=questions+reduced_test2('AM',54,3)
questions=questions+reduced_test2('LC',57,4)
questions=questions+reduced_test2('MC',61,4)
questions=questions+reduced_test2('ND',65,2)
questions=questions+reduced_test2('PfW',67,3)
questions=questions+reduced_test2('SE',70,4)

C:\Users\JBSCHOLOR-2020-18\AppData\Local\Temp\ipykernel_1020\3997413357.py:69: RuntimeWarning: invalid value encountered in double_scalars
  s=(b-a)/max(a,b)


In [74]:
indices=[]
for i in data.index:
    if(data['Code'][i] in questions):
        indices.append(i)
print(indices)

[2, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15, 16, 18, 21, 22, 23, 25, 26, 27, 28, 30, 31, 33, 35, 36, 37, 38, 40, 42, 44, 46, 48, 50, 52, 53, 54, 56, 58, 60, 61, 62, 64, 66, 67, 69, 70, 71, 74, 75, 77, 80, 81, 83, 84, 87, 90, 91, 93, 94, 97, 100, 101, 103, 104, 107, 110, 111, 112, 113, 115, 117, 120, 122]


In [75]:
X=compute_features(data,mapping)

In [76]:
X.shape

(4546, 73)

In [77]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.1, random_state=42)
svm_classifier = SVC(kernel='rbf', C=1.0, random_state=42)
svm_classifier.fit(X_train, y_train)
y_pred = svm_classifier.predict(X_train)
print("Support Vector Machines: ")
print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
y_pred = svm_classifier.predict(X_test)
print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))
random_forest_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
random_forest_classifier.fit(X_train, y_train)
y_pred = random_forest_classifier.predict(X_train)
print("Random Forest Classifier :")
print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
y_pred = random_forest_classifier.predict(X_test)
print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))
logistic_regression_classifier = LogisticRegression(max_iter=1000, random_state=42)
logistic_regression_classifier.fit(X_train, y_train)
y_pred = logistic_regression_classifier.predict(X_train)
print("Logistic Regression Classifier :")
print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
y_pred = logistic_regression_classifier.predict(X_test)
print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))
mlp = MLPClassifier(hidden_layer_sizes=(100,50), max_iter=500, random_state=42)
mlp.fit(X_train, y_train)
y_pred = mlp.predict(X_train)
print("Multi Layer Perceptron :")
print("Training accuracy: "+str(accuracy_score(y_train, y_pred)))
y_pred = mlp.predict(X_test)
print("Testing accuracy: "+str(accuracy_score(y_test, y_pred)))

Support Vector Machines: 
Training accuracy: 0.8005377658274261
Testing accuracy: 0.8285714285714286
Random Forest Classifier :
Training accuracy: 1.0
Testing accuracy: 0.8175824175824176
Logistic Regression Classifier :
Training accuracy: 0.692006844292349
Testing accuracy: 0.7274725274725274
Multi Layer Perceptron :
Training accuracy: 0.9970667318504033
Testing accuracy: 0.7208791208791209


NameError: name 'pd' is not defined